In [ ]:
#!pip install tensorflow-gpu==1.15.0
#!pip install keras==2.2.5
#!pip install h5py==2.10.0 --force-reinstall
!pip install mrcnn
!pip install flask-ngrok

#clonamos las carpetas que necesitamos.
!git clone https://github.com/matterport/Mask_RCNN.git
%cd Mask_RCNN/
!python setup.py install
!wget https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5

%cd ..
!git clone https://github.com/suso86/Deteccion-Automatica-Pinturas.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==1.15.0 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.4.0, 2.4.1, 2.4.2, 2.4.3, 2.4.4, 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.12.0)
ERROR: No matching distribution found for tensorflow-gpu==1.15.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.2/336.2 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 KB 3.7 MB/s eta 0:00:00
  Attempting uninstall: keras

In [ ]:
%cd /content/

/content


In [ ]:
!pip uninstall -y numpy
!pip install numpy==1.8.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.3 MB 7.5 MB/s 
  ERROR: Failed building wheel for numpy
  Running setup.py clean for numpy
Failed to build numpy
    Running setup.py install for numpy ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-g_e6g0pp/numpy_e6eeedadfd044cb4bc839b50321551ae/setup.py'"'"'; __file__='"'"'/tmp/pip-install-g_e6g0pp/numpy_e6eeedadfd044cb4bc839b50321551ae/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-kwwtopqe/install-record.txt --single-version-externally-managed --compile --install-headers /usr/loc

In [ ]:
import numpy
print(numpy.__version__)

1.21.6


In [ ]:
from flask.helpers import send_from_directory
!pip install pyngrok==4.1.1
!ngrok authtoken "2GZuKvg2gVgoOnoWyyTog01UDEY_54RgkWDSxGMhcGEVf4Vpp"

from flask import Flask, render_template, request, redirect, url_for, make_response,jsonify
from werkzeug.utils import secure_filename
from os import remove
import os
import cv2
from flask_ngrok import run_with_ngrok
from datetime import timedelta
import deteccion_daños_colab
from keras import backend as K

K.clear_session()

#Configuración de archivos permitidos.
ALLOWED_EXTENSIONS = set(['png','jpg', 'jpeg' ,'JPG','PNG'])

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.',1)[1] in ALLOWED_EXTENSIONS

app = Flask(__name__)
run_with_ngrok(app)

#Establecer el tiempo de caducidad de la caché de archivos estáticos
app.send_file_max_age_default = timedelta(seconds=1)

@app.route("/")
def index():
    return render_template("index.html")

@app.route("/home")
def SubirImagen():
    return render_template("formImagen.html")

@app.route("/home/daño", methods = ["POST"])
def deteccion():
    if request.method == "POST":

        f = request.files['fichero']
        name_file = f.filename

        if not(f and allowed_file(f.filename)):
            return jsonify ({"error": 1001, "msg": "Verifique el tipo de imagen cargada, solo png, jpg, jpeg, JPG, PNG"})

        #name_file = request.form.get["nombre"]
        # Ruta del archivo actual
        basepath = '/content'

        carpeta_imagenes = basepath + '/static/images/'
        upload_path = os.path.join(carpeta_imagenes , f.filename)

        f.save(upload_path)

        img = cv2.imread(upload_path)

        #remove(upload_path)

        upload_curren = os.path.join(carpeta_imagenes ,'test.jpg' )
        cv2.imwrite(upload_curren, img)

        return render_template("deteccion_daño.html", name_file=name_file)

@app.route('/home/cuadro/daños')
def MostrarDaños():
    K.clear_session()
    nombre = request.args.get('nombre')

    basepath = '/content'

    # Como estamos en windows hay que invertir los caracteres "/"
    carpeta_imagenes = basepath + '/static/images/'
    ruta_cuadro = carpeta_imagenes+nombre


    # Una vez que tenemos la imagen detectamos los daños
    zonas = deteccion_daños_colab.guardarImagen_ObtenerCaracteristicas(nombre)

    tamaño_imagen = zonas[0]
    zona_no_dañada = zonas[1]
    zona_dañada = zonas[2]
    porcentaje = zonas[3]

    remove(ruta_cuadro)

    return render_template("mostrar_daños.html",name_file=nombre,tamaño=tamaño_imagen,zona_no_dañada=zona_no_dañada,
                           zona_dañada=zona_dañada,porcentaje=porcentaje)

@app.route('/home/cuadro/daños/download')
def DownloadImages():
  pathfile = '/content/static/images/'
  path='deteccion.jpg'
  return send_from_directory(pathfile,path,as_attachment=True)


app.run()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15982 sha256=acebda4937f3e57a9f6e6cb219674114c72992362414a2ca86b473a491e387d5
  Stored in directory: /root/.cache/pip/wheels/5e/0a/51/8cb053ccd84481dd3233eba4cdb608bc7a885fd8ca418c0806
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


ModuleNotFoundError: ignored